In [1]:
!pip3 install catboost
!pip3 install hyperopt==0.2.5

     |████████████████████████████████| 76.1 MB 1.2 MB/s 
     |████████████████████████████████| 965 kB 8.9 MB/s 
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.1.2
    Uninstalling hyperopt-0.1.2:
      Successfully uninstalled hyperopt-0.1.2


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import hyperopt
from numpy.random import RandomState
from catboost.utils import select_threshold

In [8]:
csv_path = "drive/MyDrive/trades-df-2021_11_1_0_0_0-2021_12_21_0_0_0.csv"
df = pd.read_csv(csv_path)

In [9]:
x_columns = list(filter(lambda column: not 'punch' in column, df.columns))
y_columns = list(filter(lambda column: 'punch' in column, df.columns))
print(x_columns)
print(y_columns)

['trade_volume-BUY-6-sec', 'moving_average_price-BUY-6-sec', 'weighted_moving_average_price-BUY-6-sec', 'exp_moving_average_price-BUY-6-sec', 'stochastic_oscillator-BUY-6-sec', 'trade_volume-BUY-12-sec', 'moving_average_price-BUY-12-sec', 'weighted_moving_average_price-BUY-12-sec', 'exp_moving_average_price-BUY-12-sec', 'stochastic_oscillator-BUY-12-sec', 'trade_volume-BUY-18-sec', 'moving_average_price-BUY-18-sec', 'weighted_moving_average_price-BUY-18-sec', 'exp_moving_average_price-BUY-18-sec', 'stochastic_oscillator-BUY-18-sec', 'trade_volume-BUY-24-sec', 'moving_average_price-BUY-24-sec', 'weighted_moving_average_price-BUY-24-sec', 'exp_moving_average_price-BUY-24-sec', 'stochastic_oscillator-BUY-24-sec', 'trade_volume-BUY-30-sec', 'moving_average_price-BUY-30-sec', 'weighted_moving_average_price-BUY-30-sec', 'exp_moving_average_price-BUY-30-sec', 'stochastic_oscillator-BUY-30-sec', 'trade_volume-BUY-36-sec', 'moving_average_price-BUY-36-sec', 'weighted_moving_average_price-BUY-36

In [10]:
X = df[x_columns]
secs = list(sorted(list(set(map(lambda x : int(''.join(x)), [[s for s in column if s.isdigit()] for column in x_columns])))))
delta_sec = secs[1] - secs[0]
max_sec = secs[-1]
min_sec = secs[0]

for column in x_columns:
    name, side, sec, _ = column.split('-')
    if not 'price' in name and int(sec) != min_sec:
        X[f'{name}-{side}-{sec}-sec'] -= X[f'{name}-{side}-{str(int(sec) - delta_sec)}-sec']
    elif 'price' in name:
        if int(sec) != max_sec:
            X[f'{name}-{side}-{sec}-sec'] /= X[f'{name}-{side}-{str(int(sec) + delta_sec)}-sec']
        else:
            X = X.drop(f'{name}-{side}-{sec}-sec', axis=1)

X

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,trade_volume-BUY-6-sec,moving_average_price-BUY-6-sec,weighted_moving_average_price-BUY-6-sec,exp_moving_average_price-BUY-6-sec,stochastic_oscillator-BUY-6-sec,trade_volume-BUY-12-sec,moving_average_price-BUY-12-sec,weighted_moving_average_price-BUY-12-sec,exp_moving_average_price-BUY-12-sec,stochastic_oscillator-BUY-12-sec,trade_volume-BUY-18-sec,moving_average_price-BUY-18-sec,weighted_moving_average_price-BUY-18-sec,exp_moving_average_price-BUY-18-sec,stochastic_oscillator-BUY-18-sec,trade_volume-BUY-24-sec,moving_average_price-BUY-24-sec,weighted_moving_average_price-BUY-24-sec,exp_moving_average_price-BUY-24-sec,stochastic_oscillator-BUY-24-sec,trade_volume-BUY-30-sec,moving_average_price-BUY-30-sec,weighted_moving_average_price-BUY-30-sec,exp_moving_average_price-BUY-30-sec,stochastic_oscillator-BUY-30-sec,trade_volume-BUY-36-sec,moving_average_price-BUY-36-sec,weighted_moving_average_price-BUY-36-sec,exp_moving_average_price-BUY-36-sec,stochastic_oscillator-BUY-36-sec,trade_volume-BUY-42-sec,moving_average_price-BUY-42-sec,weighted_moving_average_price-BUY-42-sec,exp_moving_average_price-BUY-42-sec,stochastic_oscillator-BUY-42-sec,trade_volume-BUY-48-sec,moving_average_price-BUY-48-sec,weighted_moving_average_price-BUY-48-sec,exp_moving_average_price-BUY-48-sec,stochastic_oscillator-BUY-48-sec,...,weighted_moving_average_price-SELL-12-sec,exp_moving_average_price-SELL-12-sec,stochastic_oscillator-SELL-12-sec,trade_volume-SELL-18-sec,moving_average_price-SELL-18-sec,weighted_moving_average_price-SELL-18-sec,exp_moving_average_price-SELL-18-sec,stochastic_oscillator-SELL-18-sec,trade_volume-SELL-24-sec,moving_average_price-SELL-24-sec,weighted_moving_average_price-SELL-24-sec,exp_moving_average_price-SELL-24-sec,stochastic_oscillator-SELL-24-sec,trade_volume-SELL-30-sec,moving_average_price-SELL-30-sec,weighted_moving_average_price-SELL-30-sec,exp_moving_average_price-SELL-30-sec,stochastic_oscillator-SELL-30-sec,trade_volume-SELL-36-sec,moving_average_price-SELL-36-sec,weighted_moving_average_price-SELL-36-sec,exp_moving_average_price-SELL-36-sec,stochastic_oscillator-SELL-36-sec,trade_volume-SELL-42-sec,moving_average_price-SELL-42-sec,weighted_moving_average_price-SELL-42-sec,exp_moving_average_price-SELL-42-sec,stochastic_oscillator-SELL-42-sec,trade_volume-SELL-48-sec,moving_average_price-SELL-48-sec,weighted_moving_average_price-SELL-48-sec,exp_moving_average_price-SELL-48-sec,stochastic_oscillator-SELL-48-sec,trade_volume-SELL-54-sec,moving_average_price-SELL-54-sec,weighted_moving_average_price-SELL-54-sec,exp_moving_average_price-SELL-54-sec,stochastic_oscillator-SELL-54-sec,trade_volume-SELL-60-sec,stochastic_oscillator-SELL-60-sec
0,0.100,1.000000,1.000000,1.000000,0.0,0.400,1.000078,1.000113,1.000005,0.000000,10.355,1.000006,1.000000,1.000000,1.000000,0.500,1.000013,1.000001,1.000000,0.000000,10.655,1.000020,1.000002,1.0,1.000000,0.716,1.000023,1.000004,1.000000,0.000000,10.855,1.000019,1.000004,1.000000,1.000000,0.916,1.000000,1.000000,1.0,0.000000,...,1.000067,1.000007,0.000000,0.400,1.000035,1.000038,1.000000,1.000000,0.800,1.000006,1.000005,1.000000,0.000000,0.600,1.000035,1.000140,1.0,1.000000,3.500,1.000020,1.000004,1.000000,0.000000,0.800,1.000023,1.000024,1.0,1.000000,5.566,1.000012,1.000014,1.0,0.000000,3.400,1.000028,1.000018,1.0,1.000000,15.547,0.000000
1,0.200,1.000000,1.000000,1.000000,0.0,0.300,1.000000,1.000000,1.000000,0.000000,0.400,1.000000,1.000000,1.000000,0.000000,0.600,1.000000,1.000000,1.000000,0.000000,0.650,1.000000,1.000000,1.0,0.000000,0.800,1.000037,1.000048,1.000000,0.000000,1.399,1.000005,1.000004,1.000000,0.600000,0.900,0.999985,0.999986,1.0,0.000000,...,1.000000,1.000000,0.000000,0.766,1.000000,1.000000,1.000000,0.000000,4.084,1.000000,1.000000,1.000000,0.000000,0.766,1.000008,1.000003,1.0,0.000000,4.284,1.000025,1.000010,1.000000,0.600000,1.166,1.000000,1.000000,1.0,0.000000,4.284,0.999875,0.999760,1.0,0.600000,34.697,0.999995,1.000000,1.0,-0.339130,4.384,0.600000
2,0.200,1.0

In [50]:
commision = 0.0005
punch = commision * 5

y = pd.DataFrame(
    df[y_columns].apply(
        lambda row : 1 if max(list(row.to_numpy()), key=abs) > punch else -1 if max(list(row.to_numpy()), key=abs) < -punch else 0,
        axis = 1
    )
)
y = y.rename({0:'target'}, axis='columns')
print(y)
print(y.value_counts())

       target
0           0
1           0
2           1
3           0
4           0
...       ...
38742       0
38743       0
38744       0
38745       0
38746       0

[38747 rows x 1 columns]
target
 0        37612
-1          599
 1          536
dtype: int64


In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle = True, random_state=12)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.75, shuffle = True, random_state=6)

In [ ]:
def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=300,
        eval_metric=metrics.Accuracy(),
        random_seed=126,
        verbose=False,
        loss_function=metrics.MultiClass(),
    )
    
    cv_data = cv(
        Pool(X, y),
        model.get_params(),
        fold_count=4,
        shuffle=True,
        logging_level='Silent',
        stratified=True,
    )

    best_accuracy = np.max(cv_data['test-Accuracy-mean'])
    
    return 1 - best_accuracy


params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=1,
    trials=trials,
    rstate=RandomState(126)
)

print(best)

  0%|          | 0/3 [00:37<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [52]:
params = {
    'iterations': 500,
    #'l2_leaf_reg': int(best['l2_leaf_reg']),
    #'learning_rate': best['learning_rate'],
    'l2_leaf_reg': 2,
    'learning_rate': 0.1,
    'custom_loss': [metrics.Accuracy()],
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'depth': 5,
    'logging_level': 'Silent',
    'loss_function': 'MultiClass',
}
train_pool = Pool(X_train, y_train)
validate_pool = Pool(X_validation, y_validation)

model = CatBoostClassifier(**params)
model.fit(
    train_pool,
    eval_set=validate_pool,
    plot=True,
    verbose=False,
)

print('Model validation accuracy: {:.4}'.format(
    accuracy_score(y_test, model.predict(X_test))
))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Model validation accuracy: 0.9712


In [53]:
predictions = model.predict(X_test)
predictions = predictions.reshape(predictions.shape[0], 1)
predictions_probs = model.predict_proba(X_test)
print(predictions[90:100])
print(predictions_probs[90:100])
print(y_test[90:100])

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
[[0.32415252 0.40892479 0.26692269]
 [0.02090885 0.95814936 0.02094179]
 [0.01078654 0.9757595  0.01345396]
 [0.00789274 0.98422891 0.00787835]
 [0.01929585 0.96591847 0.01478568]
 [0.00986332 0.98084384 0.00929284]
 [0.04086921 0.92393035 0.03520044]
 [0.00889447 0.98221265 0.00889288]
 [0.00741744 0.98485587 0.0077267 ]
 [0.01466305 0.97058442 0.01475252]]
       target
20233      -1
18782       0
26657       0
27456       0
36872       0
25891       0
33455      -1
23349       0
4176        0
37949       0


In [54]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))


trade_volume-SELL-42-sec: 5.61537751161643
trade_volume-SELL-60-sec: 4.614888485896265
moving_average_price-SELL-6-sec: 4.063784580178312
weighted_moving_average_price-SELL-6-sec: 4.057274504942314
trade_volume-BUY-54-sec: 3.850347617669889
trade_volume-SELL-30-sec: 3.783192856435399
trade_volume-SELL-54-sec: 3.206172151494135
moving_average_price-BUY-12-sec: 3.198805892896194
moving_average_price-SELL-30-sec: 2.8126406799900683
trade_volume-BUY-24-sec: 2.6335753268889226
trade_volume-BUY-48-sec: 2.5656222195180702
moving_average_price-BUY-54-sec: 2.391430188770366
stochastic_oscillator-SELL-6-sec: 2.3841119242833875
weighted_moving_average_price-SELL-30-sec: 2.3002760217322757
trade_volume-SELL-36-sec: 2.1274466634340454
weighted_moving_average_price-BUY-12-sec: 2.0008079247444104
moving_average_price-BUY-36-sec: 1.9371604188044431
trade_volume-BUY-42-sec: 1.911181910669554
trade_volume-BUY-12-sec: 1.8324460643409621
trade_volume-BUY-18-sec: 1.822222371608947
moving_average_price-BUY-

In [55]:
results = []
def f(prob_threshold, diff_threshold, null_threshold):
    p = np.array((list(map(lambda row:
                          1
                          if row[2] > prob_threshold
                              # and row[2] == max(row)
                              and row[2] > row[0]
                              and row[1] < null_threshold
                              and abs(row[2] - row[0]) > diff_threshold
                              and abs(row[2] - row[1]) > diff_threshold
                          else
                          -1
                          if row[0] > prob_threshold
                              # and row[0] == max(row)
                              and row[0] > row[2]
                              and row[1] < null_threshold
                              and abs(row[0] - row[2]) > diff_threshold
                              and abs(row[0] - row[1]) > diff_threshold
                          else 0,
                          predictions_probs))))
    result = {
        'correct_predictions': 0,
        'false_positive': 0,
        'false_negative': 0,
        'wrong_side': 0,
    }
    for i in range(len(p)):
        t = int(y_test['target'][i:i+1])
        if p[i] == t != 0:
            result['correct_predictions'] += 1
        elif p[i] != t and t != 0 and p[i] != 0:
            result['wrong_side'] += 1
        elif p[i] != 0 and t == 0:
            result['false_positive'] += 1
        elif p[i] == 0 and t != 0:
            result['false_negative'] += 1

    result['correct_predictions'] /= len(y_test)
    result['false_positive'] /= len(y_test)
    result['false_negative'] /= len(y_test)
    result['wrong_side'] /= len(y_test)
    result['prob_threshold'] = prob_threshold
    result['diff_threshold'] = diff_threshold
    result['null_threshold'] = null_threshold
    results.append(result)
    return p

for null_threshold in np.arange(0.5, 0.9, 0.1):
    for prob_threshold in np.arange(0.1, 0.7, 0.1):
        for diff_threshold in np.arange(0.00, 0.20, 0.05):
            f(prob_threshold, diff_threshold, null_threshold)

In [56]:
results = list(sorted(results, key=lambda result: -calculate_profit(result['prob_threshold'], result['diff_threshold'], result['null_threshold'])))

In [57]:
print(results[0])

{'correct_predictions': 0.00012903225806451613, 'false_positive': 0.0, 'false_negative': 0.029161290322580646, 'wrong_side': 0.0, 'prob_threshold': 0.5, 'diff_threshold': 0.0, 'null_threshold': 0.5}


In [58]:
prob_threshold = results[0]['prob_threshold']
diff_threshold = results[0]['diff_threshold']
null_threshold = results[0]['null_threshold']

print(f"prob_threshold {prob_threshold}, diff_threshold {diff_threshold}, null_threshold {null_threshold}")
p = f(prob_threshold, diff_threshold, null_threshold)
print('accuracy', accuracy_score(p, y_test))
result = {
    'correct_predictions': 0,
    'false_positive': 0,
    'false_negative': 0,
    'wrong_side': 0,
}
for i in range(len(p)):
    t = int(y_test['target'][i:i+1])
    if p[i] == t != 0:
        result['correct_predictions'] += 1
    elif p[i] != t and t != 0 and p[i] != 0:
        result['wrong_side'] += 1
    elif p[i] != 0 and t == 0:
        result['false_positive'] += 1
    elif p[i] == 0 and t != 0:
        result['false_negative'] += 1

result['correct_predictions_pc'] = result['correct_predictions'] / len(y_test)
result['false_positive_pc'] = result['false_positive'] / len(y_test)
result['false_negative_pc'] = result['false_negative'] / len(y_test)
result['wrong_side_pc'] = result['wrong_side'] / len(y_test)
print(result)

prob_threshold 0.5, diff_threshold 0.0, null_threshold 0.5
accuracy 0.9708387096774194
{'correct_predictions': 1, 'false_positive': 0, 'false_negative': 226, 'wrong_side': 0, 'correct_predictions_pc': 0.00012903225806451613, 'false_positive_pc': 0.0, 'false_negative_pc': 0.029161290322580646, 'wrong_side_pc': 0.0}


In [59]:
def calculate_profit(prob_threshold, diff_threshold, null_threshold):
    p = f(prob_threshold, diff_threshold, null_threshold)
    result = {
        'correct_predictions': 0,
        'false_positive': 0,
        'false_negative': 0,
        'wrong_side': 0,
    }
    for i in range(len(p)):
        t = int(y_test['target'][i:i+1])
        if p[i] == t != 0:
            result['correct_predictions'] += 1
        elif p[i] != t and t != 0 and p[i] != 0:
            result['wrong_side'] += 1
        elif p[i] != 0 and t == 0:
            result['false_positive'] += 1
        elif p[i] == 0 and t != 0:
            result['false_negative'] += 1

    result['correct_predictions_pc'] = result['correct_predictions'] / len(y_test)
    result['false_positive_pc'] = result['false_positive'] / len(y_test)
    result['false_negative_pc'] = result['false_negative'] / len(y_test)
    result['wrong_side_pc'] = result['wrong_side'] / len(y_test)
    
    return (
          len(y) * result['correct_predictions_pc'] * (punch - 2 * commision)
          -
          len(y) * result['false_positive_pc'] * 2 * commision
          -
          len(y) * result['wrong_side_pc'] * (punch + 2 * commision)
    )
profit = calculate_profit(prob_threshold, diff_threshold, null_threshold)
print(
    'profit from 3000$ fot 2 months',
    profit * 3000,
    '$',
    profit * 100,
    '%'
)

profit from 3000$ fot 2 months 22.498258064516133 $ 0.7499419354838711 %


In [60]:
correct_accuracy = result['correct_predictions'] / len(y_test[y_test['target'] != 0])
print(
f"Correct accuracy = correct prediction / total positive num = {correct_accuracy}"
)
FPR = result['false_positive'] / len(y_test[y_test['target'] == 0])
print(
f"FPR = false positive / total negative num = {FPR}"
)
FNR = result['false_negative'] / len(y_test[y_test['target'] != 0])
print(
f"FNR = false negative / total positive num = {FNR}"
)
cases_pc = (result['correct_predictions'] + result['false_positive'] + result['wrong_side']) / len(y_test[y_test['target'] != 0]) * 100
probability_pc = result['correct_predictions'] / (result['correct_predictions'] + result['false_positive'] + result['wrong_side']) * 100
print(
f"Model in {cases_pc}% of cases with probability {probability_pc}% predict a correct market jump"
)

Correct accuracy = correct prediction / total positive num = 0.004405286343612335
FPR = false positive / total negative num = 0.0
FNR = false negative / total positive num = 0.9955947136563876
Model in 0.4405286343612335% of cases with probability 100.0% predict a correct market jump


In [61]:
for i in range(len(predictions_probs)):
    if int(y_test['target'][i:i+1]) != 0:
        print(p[i], predictions_probs[i], int(y_test['target'][i:i+1]))

0 [0.01002155 0.97815069 0.01182777] 1
0 [0.01673986 0.95929627 0.02396386] -1
0 [0.01051587 0.97989528 0.00958886] -1
0 [0.01344922 0.97260521 0.01394558] -1
0 [0.35531848 0.45688737 0.18779415] 1
0 [0.32415252 0.40892479 0.26692269] -1
0 [0.04086921 0.92393035 0.03520044] -1
0 [0.01487971 0.96950016 0.01562014] 1
0 [0.04639228 0.91974886 0.03385886] -1
0 [0.14735192 0.76154143 0.09110665] 1
0 [0.02587967 0.95308716 0.02103317] 1
0 [0.07277492 0.86227077 0.0649543 ] -1
0 [0.039144   0.91564476 0.04521125] 1
0 [0.01170684 0.97821686 0.0100763 ] -1
0 [0.03616034 0.93718055 0.02665911] 1
0 [0.02127823 0.95861301 0.02010877] 1
0 [0.00998267 0.98051399 0.00950333] -1
0 [0.39602934 0.3052214  0.29874927] 1
0 [0.04671071 0.92481223 0.02847706] 1
0 [0.02604175 0.9502295  0.02372876] 1
0 [0.42657889 0.35029494 0.22312618] 1
0 [0.01211554 0.97563915 0.01224531] 1
0 [0.01085525 0.97876194 0.0103828 ] -1
0 [0.01440787 0.97095261 0.01463953] 1
0 [0.04182415 0.92461397 0.03356188] 1
0 [0.02572101 0